In [1]:
avg_rating_csv = "hdfs:///user/cloudera/data/input/avg_rating.csv"
input_meta = "hdfs:///user/cloudera/data/input/meta_Electronics.json"  
output = "hdfs:///user/cloudera/data/output_spark_prod_title"

In [2]:
def get_dict_from_csv(csv_row):
    result = list(csv.reader([csv_row]))
    return (result[0][0], result[0][1])

In [3]:
import ast

def get_prod_id_and_title(product_json_item):
    dict_product_item = ast.literal_eval(product_json_item)
    return (dict_product_item['asin'], dict_product_item.get('title', None))

In [4]:
import csv, io

def list_to_csv_str(x):
    """Given a list of strings, returns a properly-csv-formatted string."""
    output = io.StringIO("")
    csv.writer(output).writerow(x)
    return output.getvalue().strip() # remove extra newline

In [5]:
rdd_avg_ratings = sc.textFile(avg_rating_csv).map(lambda row: get_dict_from_csv(row))

In [6]:
rdd_titles = sc.textFile(input_meta).map(lambda row: get_prod_id_and_title(row)).filter(lambda x: x[1] is not None)

In [7]:
rdd_id_title_rating = rdd_avg_ratings.join(rdd_titles).map(lambda x: (x[0], "\"" + x[1][1] + "\"", x[1][0]))

In [8]:
rdd_id_title_rating = rdd_id_title_rating.filter(lambda x: x[0] is not None and x[1] is not None and x[2] is not None)

In [20]:
rdd_id_title_rating.map(list_to_csv_str).saveAsTextFile(output)

In [9]:
rdd_id_title_rating.map(list_to_csv_str).take(5)

['B00000JBUI,"""Logitech TrackMan Marble FX Trackball""",4.909090909090909',
 'B00001W0DG,"""Sony MDR-V500DJ Monitor Series Headphones with Swivel Earcups (Discontinued by Manufacturer)""",3.6666666666666665',
 'B00003CWG4,"""Monster Cable TGHZ-4RF 2 Gigahertz 4-Way Low-Loss RF Splitters for TV &amp; Satellite (Discontinued by Manufacturer)""",4.0',
 'B00004TZE5,"""CD30 Nylon CD Case (30-Capacity) (Discontinued by Manufacturer)""",4.625',
 'B00004WLJ5,"""Gardner Bender 46-310 Electrical 11-Inch Cable Ties, 100-Pack, Natural""",4.411764705882353']